랭체인을 위한 모듈: 

랭체인에는 많은 모듈들이 있다.

모델 IO - 모듈에는 입력input과 출력output이 있다.
입력은 prompt(명령을 내리는 곳)이다.
따라서 모델 IO에는 prompt template, 언어 모델, LMS 또는 채팅 모델, output parser 등이 있다.

Retrieval - 외부 데이터로 접근하여 이를 모델에 어떻게 제공하는 것에 관한 것.  
document loaders, transformers, text embedding, vector stores와 retrievers 등이 있다.  
이것들 전부 우리의 데이터들로 작업하게 하고 모델에게 제공할 수 있는지에 관한 것이다.  

Chains - 

Agents - 독립적으로 AI가 작동하도록 만들수 있게 해주는 것.
chains이 필요한 도구들을 선택하여 사용할 수 있도록!  
일을 chains에 시키고 그에 맞는 커스텀 도구를 만들어 준다면 chains 스스로 사용할 tool들을 선택한다는 것이다.  

Memory - 챗봇에 memory(기억)할 수 있도록 하는 것.

callbacks - model이 무엇을 하고 있는지 중간에 알 수 있도록 하는 것.  
모델이 답변을 제공하기 전에 실제로 어떤 일을 하고 있는 지 확인할 수 있다.

다음 모듈들 중 모델 IO에 대해서 알아보자.  
model I/O 이 편해져야 하고 특징들도 알아봐야 하고 다른 종류의 메모리에 대해서도 알아야 한다.   

먼저 FewShotPromptTemplate 에 대해서 알아보자

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
# FewShotPromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

# 일반적인 template 만들기 - template 를 만들면 유효성 검사를 할 수 있어서 좋다.
template = PromptTemplate.from_template("What is the capital of {country}")
template.format(country="France")

'What is the capital of France'

template를 사용하는 이유는 유효성 검사를 할 수 있기 때문이다.  
그리고 다른 하나는 prompt template를 디스크에 저장하고 load 할 수 있기 때문이다.  

규모가 큰 language model을 만들기 시작할 때 prompt는 정말 중요하다.  
데이터베이스나 파일 등에서 prompt를 로드해야 할 일도 생길 수 있다.  

위에서 작성했던 PromptTemplate.from_template("What is the capital of {country}") 는 마치 단축키와 같다.  
그래서 이런 식으로 하지 않으면 어떤 식으로 해야하는지 한번 살펴보자.  

먼저 PromptTemplate에 template를 전달해준다.  
그런 다음 입력 변수(input variables)들을 전달해준다.  



In [ ]:
template = PromptTemplate(
    template="What is the capital of {country}",
    input_variables=["country"],
    
)
template.format(country="France")

'What is the capital of France'

위 방법이나 아래 방법이나 결과는 똑같지만 위 방법이 훨씬 더 좋다.  
이런식으로 하면 자동으로 랭체인이 어떤 변수가 우리에게 필요한 지 알아서 계산해서 유효성 검사를 진행한다.

---
이제 FewShotPromptTemplate와 FewShot Learning이 무엇인지 알아보자.  
기본적으로 FewShot은 모델에게 예제들을 준다는 뜻과 같다.  
더 나은 대답을 할 수 있도록 하는 예제들을 제공해 준다는 것.  

만약 구체적으로 대답하는 AI 모델이 필요하다가 가정해보면   
모델에게 어떻게 대답해야 하는지에 대한 예제를 AI 모델에게 주는 것이   
prompt를 사용해서 어떻게 대답해야 하는지 알려주느 것보다 훨씬 좋다.  (일일이 써서 보여주는 것보다 예제를 보여줘라!)

이것이 fewShotPromptTemplate가 하는 일이다.




먼저 예제들을 리스트로 만들어보자

In [ ]:
examples = [
    {
        "question":"what do you know about Greece?",
        "answer":"""
        I know this: 
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
    {
        "question":"what do you know about Italy?",
        "answer":"""
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question":"what do you know about south korea?",
        "answer":"""
        I know this:
        Capital: Seoul
        Language: Korean
        Food: Kimchi and Galbi
        Currency: won
        """,
    },
]

위 예제들은 기본적으로 모델이 나에게 이런 식으로 답변해줬으면 좋겠다 하는 예제들이다.

만약 예제를 주지 않고 LLM 에게 물어본다면?

In [ ]:
chat.predict("what do you know about Greece?")

Greece is a country located in southeastern Europe, known for its rich history, ancient civilization, and stunning landscapes. It is considered the cradle of Western civilization, as it was the birthplace of democracy, philosophy, and the Olympic Games.

Greece is famous for its ancient ruins, including the Acropolis in Athens, the Temple of Apollo in Delphi, and the Palace of Knossos in Crete. The country is also known for its beautiful islands, such as Santorini, Mykonos, and Rhodes, which attract millions of tourists each year.

Greek cuisine is also popular worldwide, with dishes like moussaka, souvlaki, and baklava being enjoyed by people all over the world. The country is also known for its vibrant culture, including traditional music, dance, and festivals.

Greece has faced economic challenges in recent years, but it remains a popular destination for travelers seeking history, culture, and natural beauty.

'Greece is a country located in southeastern Europe, known for its rich history, ancient civilization, and stunning landscapes. It is considered the cradle of Western civilization, as it was the birthplace of democracy, philosophy, and the Olympic Games.\n\nGreece is famous for its ancient ruins, including the Acropolis in Athens, the Temple of Apollo in Delphi, and the Palace of Knossos in Crete. The country is also known for its beautiful islands, such as Santorini, Mykonos, and Rhodes, which attract millions of tourists each year.\n\nGreek cuisine is also popular worldwide, with dishes like moussaka, souvlaki, and baklava being enjoyed by people all over the world. The country is also known for its vibrant culture, including traditional music, dance, and festivals.\n\nGreece has faced economic challenges in recent years, but it remains a popular destination for travelers seeking history, culture, and natural beauty.'

다음은 FewShotPromptTemplate를 사용해서 만들어 보자.


첫 번째 단계는 예제의 형식을 지정하는 것이다.  
형식화 하려면 형식 지정 도구를 만들어야 한다.  
위 예제는 여기에 작성했지만(examples) 데이터베이스 등 다른곳에서 가져올 수도 있거나 할 수 있기 때문에 예제의 형식을 지정해야 한다.

우리는 human에는 질문{question}을 넣어주고 AI에는 답변{answer}을 넣어주는 형식을 만들어 보자.


In [ ]:
# 예제의 형식을 지정하는 방법
example_template = """
    Human: {question}
    AI: {answer}
"""


In [ ]:
example_prompt = PromptTemplate.from_template(example_template)

또는

In [ ]:
example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

---
이제 fewshotprompttemplate을 사용해서 실제 prompt를 만들어보자.

examples 와 example_prompt 를 넘겨준다.  
이렇게 하면 랭체인이 알아서 각각의 예제 리스트(examples)들을 이 prompt(example_prompt)를 사용하여 형식화 한다.

다음으로는 사용자의 질문이 어떻게 보여줄지 지정하는 suffix 를 작성한다.  
suffix는 형식화 된 모든 예제 마지막에 나오는 내용이다.  
위에 나와있는 {question}과 동일한 형식을 사용한다.



In [ ]:
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="what do you know about {country}?",
    input_variables=["country"]
)

In [ ]:
prompt.format(country="japan")

'\n    Human: what do you know about Greece?\n    AI: \n        I know this: \n        Capital: Athens\n        Language: Greek\n        Food: Souvlaki and Feta Cheese\n        Currency: Euro\n        \n\n\n\n    Human: what do you know about Italy?\n    AI: \n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\n\n\n    Human: what do you know about south korea?\n    AI: \n        I know this:\n        Capital: Seoul\n        Language: Korean\n        Food: Kimchi and Galbi\n        Currency: won\n        \n\n\nwhat do you know about japan?'

what do you know about japan?

위 내용은 언어 모델이 우리를 위해 딱 맞게 자동 완성시켜준 것이다.  

우리는 예제리스트(examples)를 만들었고, fewShotPromptTemplate에 전달했다.  
어떻게 전달한 예제들을 형식화 할지 알려주었고(example_prompt),  
마지막에 질문을 포함시켰다(suffix).  
그리고 유효성 검사를 받을 수 있도록, 템플릿에서 사용할 입력변수(input_variables)가 무엇인지 지정했다.

AI는 우리의 예제들과 똑같은 구조, 형태로 답변하게 될 것이다.  

이제 끝났으니 실제로 이것은 chains 안에 넣어보자.



In [ ]:
chain = prompt | chat

chain.invoke({"country":"japan"})

AI: 
        I know this:
        Capital: Tokyo
        Language: Japanese
        Food: Sushi and Ramen
        Currency: Yen

AIMessageChunk(content='AI: \n        I know this:\n        Capital: Tokyo\n        Language: Japanese\n        Food: Sushi and Ramen\n        Currency: Yen')

In [ ]:
chain.invoke({"country":"Germany"})

AI: 
        I know this:
        Capital: Berlin
        Language: German
        Food: Bratwurst and Sauerkraut
        Currency: Euro

AIMessageChunk(content='AI: \n        I know this:\n        Capital: Berlin\n        Language: German\n        Food: Bratwurst and Sauerkraut\n        Currency: Euro')

전체 코드 살펴보기

In [1]:
from langchain.chat_models import ChatOpenAI
# from langchain.prompts import PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
# FewShotPromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
# Memory


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

examples = [
    {
        "question":"what do you know about Greece?",
        "answer":"""
        I know this: 
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
    {
        "question":"what do you know about Italy?",
        "answer":"""
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question":"what do you know about south korea?",
        "answer":"""
        I know this:
        Capital: Seoul
        Language: Korean
        Food: Kimchi and Galbi
        Currency: won
        """,
    },
]

# 예제의 형식을 지정하는 방법
example_template = """
    Human: {question}
    AI: {answer}
"""

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="what do you know about {country}?",
    input_variables=["country"]
)

# chain = prompt | chat

# chain.invoke({"country":"austraila"})



NameError: name 'example_prompt' is not defined

약간 수정해보자.ㅠ

먼 예제들을 수정해보자.

먼저 example_promtp를 작성한다.

In [ ]:
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate

examples = [
    {
        "country":"Greece",
        "answer":"""
        I know this: 
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
    {
        "country":"Italy",
        "answer":"""
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country":"korea",
        "answer":"""
        I know this:
        Capital: Seoul
        Language: Korean
        Food: Kimchi and Galbi
        Currency: won
        """,
    },
]

# promptTemplate 대신 ChatPromptTemplate 을 사용
# promptTempplate.from_template() 가 아닌 ChatPromptTemplate.from_messages()를 사용한다.
# 지금 하고 있는 것은 어떻게 예제들을 형식화 할 것인가 이다.
example_prompt = ChatPromptTemplate.from_messages([
    # prompt template 안에 모든 질문이 포함되어 있다.
    ("human", "what do you know about {country}?"),
    ("ai", "{answer}")
])



우리가 지금까지 하고 있던 건 example 목록에 있는 각 예제의 형식을 지정해 주는 것이다.  

ChatPromptTemplate를 사용하여 형식을 지정해줬다.

human 이 "이 나라에 대해 무엇을 알고 있어?" 라고 묻는 거다.  
AI는 "너 이런 식으로 답변했어" 같이 답변하게 만드려고 AI를 속이고 있다고 할 수 있다.  
그러면 AI가 내가 이런식으로 답변했구나를 기억하면서 이런 식으로 말하려고 할 것이다.  

In [ ]:
# FewShotChatMessagePromptTemplate 를 사용해보자.
# 이건 suffix가 필요하지 않다.
example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt 여기에서는 시스템 메세지를 이전의 예제들과 함께 넣어줬다.


In [ ]:

# 마지막에 final_prompt 를 작성해주자.
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert"),
    example_prompt,
    ("human", "what do you know about {country}?")
    
])

In [ ]:
chain = final_prompt | chat

chain.invoke({"country": "germany"})

I know this:
Capital: Berlin
Language: German
Food: Bratwurst and Sauerkraut
Currency: Euro

AIMessageChunk(content='I know this:\nCapital: Berlin\nLanguage: German\nFood: Bratwurst and Sauerkraut\nCurrency: Euro')

---

채팅 봇을 위한 fewShotPromptTemplate 에 대해서 알아보자
왜냐하면 채팅의 특성상 상호작용 해야하는 예제들이 있기 때문.

FewShotChatMessagePromptTemplate 전체 코드 살펴보기

In [ ]:
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

examples = [
    {
        "country": "Greece",
        "answer": """
        I know this: 
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "korea",
        "answer": """
        I know this:
        Capital: Seoul
        Language: Korean
        Food: Kimchi and Galbi
        Currency: won
        """,
    },
]

# promptTemplate 대신 ChatPromptTemplate 을 사용
# promptTempplate.from_template() 가 아닌 ChatPromptTemplate.from_messages()를 사용한다.
# 지금 하고 있는 것은 어떻게 예제들을 형식화 할 것인가 이다.

# 예제 대화 형식 지정
example_prompt = ChatPromptTemplate.from_messages(
    [
        # prompt template 안에 모든 질문이 포함되어 있다.
        ("human", "what do you know about {country}?"),
        ("ai", "{answer}"),
    ]
)


# FewShotChatMessagePromptTemplate 를 사용해보자.
# 이건 suffix가 필요하지 않다.

# 여러 예제들을 하나의 프롬프트로 만듬
example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)


# 마지막에 final_prompt 를 작성해주자.

# 전체 프롬프트의 최종 구조를 정의한다.
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert"),
        example_prompt,
        ("human", "what do you know about {country}?"),
    ]
)

chain = final_prompt | chat

chain.invoke({"country": "germany"})